In [37]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [38]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
import lgpio
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle
from gripper_controller import Gripper
from micropipette_controller import Micropipette

In [39]:

class Agitador:
    def __init__(self, pin=6, chip=0):
        """Inicializa o agitador, configurando o pino GPIO."""
        self.pin = pin
        self.chip = chip
        self.h = lgpio.gpiochip_open(self.chip)
        lgpio.gpio_claim_output(self.h, self.pin)
        print(f"Agitador configurado no GPIO{self.pin}.")

    def ligar(self):
        """Liga o agitador."""
        lgpio.gpio_write(self.h, self.pin, 1)
        print("Agitador ligado.")

    def desligar(self):
        """Desliga o agitador."""
        lgpio.gpio_write(self.h, self.pin, 0)
        print("Agitador desligado.")

    def ativar_por(self, tempo):
        """Liga o agitador por um tempo em segundos."""
        self.ligar()
        time.sleep(tempo)
        self.desligar()

    def fechar(self):
        """Libera o acesso ao chip GPIO."""
        lgpio.gpiochip_close(self.h)
        print("Chip GPIO liberado.")


In [40]:
jubilee = JubileeMotionController()

In [41]:
jubilee.home_all(mesh_mode_z=False)
jubilee.protect_tools(on=True)
jubilee.move_xyz_absolute(z=50)

In [42]:
pipeta = Micropipette(jubilee,parking_position_xy=(138,16),move_velocity=3000)

In [43]:
agitador = Agitador()

error: 'GPIO busy'

In [44]:
pipeta.install()

In [45]:
jubilee.gcode("M208 Z0:320")

''

In [46]:
altura_segura_1 = 280
altura_segura_2 = 320
velocidade_movimento_xy = 6000
posicao_descarte = (105,240,270)

In [47]:
def acoplar_ponteira(altura_segura, posicao_caixa_ponteiras):
    jubilee.move_xyz_absolute(z=altura_segura)
    jubilee.move_xyz_absolute(posicao_caixa_ponteiras[0], posicao_caixa_ponteiras[1], velocity=velocidade_movimento_xy)
    jubilee.move_xyz_absolute(z=posicao_caixa_ponteiras[2], velocity=500)
    jubilee.move_xyz_absolute(z=altura_segura)
    posicao_caixa_ponteiras[0] -= 10


def descarte_ponteira():
    jubilee.move_xyz_absolute(z=altura_segura_2)
    jubilee.move_xyz_absolute(posicao_descarte[0], posicao_descarte[1], velocity=velocidade_movimento_xy)
    jubilee.move_xyz_absolute(z=posicao_descarte[2], velocity=1000)
    pipeta.eject_tip()


def pipetar_liquido(origem, destino, volume_ul, altura_segura, velocidade_dispensacao=300):
    ciclos = int(volume_ul // 1000)
    restante = volume_ul % 1000

    def transferir(volume):
        jubilee.move_xyz_absolute(z=altura_segura)
        pipeta.press(volume)
        jubilee.move_xyz_absolute(origem[0], origem[1], velocity=velocidade_movimento_xy)
        jubilee.move_xyz_absolute(z=origem[2], velocity=600)
        pipeta.aspirate()
        jubilee.move_xyz_absolute(z=altura_segura)
        jubilee.move_xyz_absolute(destino[0], destino[1], velocity=velocidade_movimento_xy)
        jubilee.move_xyz_absolute(z=destino[2],velocity=600)
        pipeta.dispense(velocity=velocidade_dispensacao)
        jubilee.move_xyz_absolute(z=altura_segura)

    for _ in range(ciclos):
        transferir(1000)

    if restante > 0:
        transferir(restante)


<p align ='justify'>
Para preparar a solução dos núcleos de ouro, 
300 μL de <em>HAuCl<sub>4</sub></em> (Sigma-Aldrich, ≥ 99.9%) 
0,01 M foram adicionados a 7,5 mL de 
<em>CTAB</em> (Sigma-Aldrich, ≥ 99.9%) 0,1 M sob agitação. 
Em seguida, foram adicionados 500 μL de 
<em>NaBH<sub>4</sub></em> (Sigma-Aldrich, ≥ 99.9%) 0,01 M resfriado 
e o sistema foi mantido sob agitação vigorosa por 10 minutos.
Após esse período, o sistema foi mantido em repouso à temperatura ambiente 
(23&nbsp;°C) e protegido da luz por 2 horas antes do uso.
</p>


In [48]:
posicao_erlenmeyer = (243,161,300)
posicao_caixa_ponteiras = [144.9,312,160]

acido_cloroaurico = (31,191,175)
nitrato_prato = (31,178,175)
acido_ascorbico = (31,165,175)
borohidreto = (31,155,175)
solucao_previa = (31,143,175)


ctab = (133,133,160)


In [ ]:
#Acoplar Primeira Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 300ul de ácido cloroáurico
pipetar_liquido(acido_cloroaurico,posicao_erlenmeyer,300,altura_segura_2,velocidade_dispensacao=300)

#Remover ponteira
descarte_ponteira()

#Acoplar Segunda Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 7500ul de CTAB
pipetar_liquido(ctab,posicao_erlenmeyer,7500,altura_segura_2,velocidade_dispensacao=300)

#Ejetar segunda ponteira
descarte_ponteira()

agitador.ligar()

#Acoplar Terceira Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 500ul de borohidreto de sódio
pipetar_liquido(borohidreto,posicao_erlenmeyer,500,altura_segura_2,velocidade_dispensacao=300)

#Ejetar terceira ponteira
descarte_ponteira()
time.sleep(900)

#Desativar Agitador
agitador.desligar()
agitador.fechar()



<p align='justify'>
Para a solução de crescimento, 
1 mL de <em>HAuCl<sub>4</sub></em> (Sigma-Aldrich, ≥ 99.9%) 0,01 M foi adicionado a 
20 mL de <em>CTAB</em> (Sigma-Aldrich, ≥ 99.9%) 0,1 M. 
Em seguida, foram adicionados 200&nbsp;μL de 
<em>AgNO<sub>3</sub></em> (Synth, P. A.) 0,01 M, seguido de 120&nbsp;μL de 
ácido ascórbico 0,01 M (Sigma-Aldrich, ≥ 99.9%) e 100&nbsp;μL da solução de núcleos 
previamente preparada.
</p>


In [ ]:
posicao_erlenmeyer = (243,161,300)
#posicao_caixa_ponteiras = [144.9,312,160]

acido_cloroaurico = (31,191,175)
nitrato_prato = (31,178,175)
acido_ascorbico = (31,165,175)
borohidreto = (31,155,175)
solucao_previa = (31,143,175)


ctab = (133,133,160)


In [ ]:
#Acoplar Primeira Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 1000ul de ácido cloroáurico
pipetar_liquido(acido_cloroaurico,posicao_erlenmeyer,1000,altura_segura_2,velocidade_dispensacao=300)

#Remover ponteira
descarte_ponteira()


#Acoplar Segunda Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 200ul de nitrato de prata
pipetar_liquido(nitrato_prato,posicao_erlenmeyer,200,altura_segura_2,velocidade_dispensacao=300)

#Remover ponteira
descarte_ponteira()


#Acoplar Terceiro Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 120ul de nitrato de prata
pipetar_liquido(acido_ascorbico,posicao_erlenmeyer,120,altura_segura_2,velocidade_dispensacao=300)

#Remover ponteira
descarte_ponteira()


#Acoplar Quarta Ponteira
acoplar_ponteira(altura_segura=altura_segura_1,posicao_caixa_ponteiras=posicao_caixa_ponteiras)

#Pipeta 100ul de seeds de ouro
pipetar_liquido(solucao_previa,posicao_erlenmeyer,120,altura_segura_2,velocidade_dispensacao=300)

#Remover ponteira
descarte_ponteira()



In [54]:
drop = (105,240,200)
pipetar_liquido(ctab,drop,1000,250,velocidade_dispensacao=10)


In [15]:
agitador = Agitador()

error: 'GPIO busy'

In [ ]:
agitador.fechar()

In [55]:
pipeta.uninstall()